You should add this link to your google drive. Click the folder. https://drive.google.com/drive/folders/1O6FVKlGHagPeg3R5exvx1SHx96biEvpG?usp=sharing .In the top right, click Add a shortcut to My Drive.

After finishing the hands on, download your notebook and submit it to the https://docs.google.com/forms/d/1s3lmt2sO_ky4RhtplMyD6I_42TGd8OA0NBnqU94P-ig/edit

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import cv2 as cv
import numpy as np
import os
from numpy import linalg as LA

def getHOGHist(ang,mag):
    feature_vector = []
    hist_keys = np.array([0, 20, 40, 60, 80, 100, 120, 140, 160])
    for i in range(0, 256, 8):
        for j in range(0, 256, 8):

            hist_vals = np.zeros(shape=hist_keys.size)

            block_dir = ang[i:i + 8, j:j + 8]

            block_mag = mag[i:i + 8, j:j + 8]

            for row in range(0, 8):
                for col in range(0, 8):
                    mag_cell = block_mag[row, col]
                    dir_cell = block_dir[row, col]

                    for ind in range(1, 9):
                        if (dir_cell <= hist_keys[ind]):
                            ratio1 = ((hist_keys[ind] - dir_cell)/ 20)
                            ratio2 = ((dir_cell - hist_keys[ind-1]) / 20)
                            comp1 = ratio1 * mag_cell
                            comp2 = ratio2 * mag_cell
                            hist_vals[ind - 1] += comp1
                            hist_vals[ind] += comp2
                            break

                    if dir_cell > 160:
                        ratio1 = ((180 - dir_cell) / 20)
                        ratio2 = ((dir_cell - 160) / 20)
                        comp1 = ratio1 * mag_cell
                        comp2 = ratio2 * mag_cell
                        hist_vals[0] += comp1
                        hist_vals[8] += comp2

            feature_vector.append(hist_vals)

    #print(len(feature_vector)) #1024 each consists of 1*9

    feature_vector = np.reshape(feature_vector, (32, 32, 9))
    features = []
    for i in range(0, 31): #31 because output_size = image_size(32) - windowSize(2) + 1 when stride is 1
        for j in range(0, 31):

            hist_window = []
            for cellIndexI in range(i, i + 2):
                for cellIndexJ in range(j, j + 2):
                    hist_cell = feature_vector[cellIndexI,cellIndexJ]
                    hist_window.append(hist_cell)

            hist_window = np.ravel(hist_window) #36*1
            norm = LA.norm(hist_window)
            hist_window = hist_window / norm

            features.append(hist_window)

    features = np.ravel(features).reshape(-1, 1)#(34596,1) which is 36*31*31

    return features


def extract_features(cap):
    video_features = []
    ret, frame1 = cap.read()
    prvs = cv.cvtColor(frame1, cv.COLOR_BGR2GRAY)
    prvs = cv.resize(prvs, (256,256))

    nFramesInCurrentVideo = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print('Number of frames in video: ', nFramesInCurrentVideo)
    RequiredFrames = 20
    frame_step = int(nFramesInCurrentVideo / RequiredFrames)
    FrameIndex = frame_step
    while FrameIndex <= nFramesInCurrentVideo:
        cap.set(cv.CAP_PROP_POS_FRAMES, FrameIndex)
        _, frame2 = cap.read()
        next = cv.cvtColor(frame2, cv.COLOR_BGR2GRAY)
        next = cv.resize(next, (256,256))
        flow = cv.calcOpticalFlowFarneback(prvs, next, None, 0.7, 4, 12, 4, 6, 1.1, 0)
        mag, ang = cv.cartToPolar(flow[..., 0], flow[..., 1])
        angle = ang * 180 / np.pi / 2
        magnitude = cv.normalize(mag, None, 0, 255, cv.NORM_MINMAX)

        currentFrameFeatures = getHOGHist(angle,magnitude)
        video_features.append(currentFrameFeatures)

        FrameIndex = FrameIndex + frame_step
        prvs = next


    cap.release()

    return video_features

In [3]:
import cv2 as cv
import numpy as np
import os
from sklearn import svm
import pickle

def train():

    all_features = []
    all_labels = []
    folder_path = "/content/drive/MyDrive/HOF/Training"
    for file_name in os.listdir(folder_path):
        cap = cv.VideoCapture(folder_path + '/' + file_name)
        video_features = extract_features(cap)
        label = 0 if "walk" in file_name else 1
        video_labels = [label] * len(video_features)
        all_features += video_features
        all_labels += video_labels

    NFrames = len(all_features)
    all_features = np.array(all_features)
    all_labels = np.array(all_labels)
    all_features = np.squeeze(all_features, axis=2)
    all_features = np.nan_to_num(all_features)
    all_labels = all_labels.reshape(NFrames, 1)
    clf = svm.SVC()
    clf.fit(all_features, all_labels)
    filename = 'trained_model.sav'
    pickle.dump(clf, open(filename, 'wb'))


def test():
    folder_path = "/content/drive/MyDrive/HOF/Testing"
    filename = "/content/drive/MyDrive/HOF/trained_model.sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    for file_name in os.listdir(folder_path):
        cap = cv.VideoCapture(folder_path + '/' + file_name)
        video_features = extract_features(cap)
        NFrames = len(video_features)
        yactual = 0 if "walk" in file_name else 1
        video_labels = [yactual] * NFrames
        video_features = np.array(video_features)
        video_labels = np.array(video_labels)
        video_features = np.squeeze(video_features, axis=2)
        video_labels = video_labels.reshape(NFrames, 1)
        video_features = np.nan_to_num(video_features)
        prediction = loaded_model.predict(video_features)
        predicted_label = np.round(np.mean(prediction))

        print("Video: " + file_name + " Predicted Label: " + str(
            predicted_label) + ", Actual Label: " + str(video_labels[0]))


#train()
test()


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Number of frames in video:  239
Video: v_jumping_01_01.mpg Predicted Label: 1.0, Actual Label: [1]
Number of frames in video:  106


<ipython-input-2-608c779f4809>:58: RuntimeWarning: invalid value encountered in divide
  hist_window = hist_window / norm


Video: v_jumping_01_04.mpg Predicted Label: 1.0, Actual Label: [1]
Number of frames in video:  239
Video: v_walk_dog_02_05.mpg Predicted Label: 0.0, Actual Label: [0]
Number of frames in video:  214
Video: v_walk_dog_15_01.mpg Predicted Label: 1.0, Actual Label: [0]
